In [ ]:
from transformers import AutoTokenizer
import pandas as pd
import glob

In [ ]:
tokenizer_base = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')
tokenizer_chat = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')

In [ ]:
tokenizer_base.get_vocab() == tokenizer_chat.get_vocab()

In [ ]:
df_TGT  = pd.read_json('../../../data/chq/test.jsonl',orient='record',lines=True)
list_rs = df_TGT['target'].to_list()

gs_base_icl = [x.splitlines()[0] for x in open('./chq_30_2_Llama_Base.txt','r').read().split('\n++++++++++++++++++\n')]
gs_base_sft = open('./CHQ_20_Base.txt','r').read().split('\n++++++++++++++++++\n')

gs_chat_icl = open('./chq_20_2_Llama_Chat.txt','r').read().split('\n++++++++++++++++++\n')
gs_chat_sft = open('./CHQ_20_Chat.txt','r').read().split('\n++++++++++++++++++\n')

In [ ]:
from collections import defaultdict
dict_cmp = defaultdict(list)

for gs,rs in zip(gs_base_icl, list_rs):
    dict_cmp[rs].append(gs)

for gs,rs in zip(gs_base_sft, list_rs):
    dict_cmp[rs].append(gs)

for gs,rs in zip(gs_chat_icl, list_rs):
    dict_cmp[rs].append(gs)

for gs,rs in zip(gs_chat_sft, list_rs):
    dict_cmp[rs].append(gs)

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
def RL(ref, hyp):
    scores = scorer.score(ref,hyp)
    return scores['rougeL'].fmeasure

In [ ]:
for rs,gss in dict_cmp.items():
  try:
    rl_base_icl = RL(rs.split('||')[0], gss[0])
    rl_base_sft = RL(rs.split('||')[0], gss[1])
    rl_chat_icl = RL(rs.split('||')[0], gss[2])
    rl_chat_sft = RL(rs.split('||')[0], gss[3])
    
    dict_cmp[rs].extend([rl_base_icl, rl_base_sft, rl_chat_icl, rl_chat_sft])
    
  except: print(rs)



In [ ]:
list_src = df_TGT['inputs'].to_list()


In [ ]:
list_src[0], gs_base_icl[0], list_rs[0]

In [ ]:
import re
pattern = r"^[A-Za-z▁]+$"

def OOV_frac(tokenizer,rs):
    tokens = rs.split()
    
    oov_frac = 0
    consider = 0
    oov_list = list()
    for token in tokens:
        if re.match(pattern,token): 
            consider += 1
            if len(tokenizer.tokenize(token)) > 1: 
                oov_frac += 1
                
    return oov_frac/consider

def Novel_frac(src,rs):
    novel_frac = 0
    novel_list = list()
    for tok in rs.split():
        if tok not in src: 
            novel_frac += 1
            novel_list.append(tok)
    
    return novel_frac/len(rs.split()), novel_list

In [ ]:
OOV_frac(tokenizer_base,list_rs[10]), list_rs[10]

In [ ]:
dict_src = defaultdict(list)
for rs,src in zip(list_rs,list_src): dict_src[rs].append(src)

list_del_key = list()
for key,val in dict_src.items():
    # print(key,val)
    if len(val[0]) == 0: 
        list_del_key.append(key)
        continue
        
    dict_src[key].extend(Novel_frac(val[0].lower(),key))
    dict_src[key].append(OOV_frac(tokenizer_base,val[0]))
    dict_src[key].append(OOV_frac(tokenizer_chat,val[0]))
    

In [ ]:
list_rl_base_icl = list()
list_rl_base_sft = list()
list_rl_chat_icl = list()
list_rl_chat_sft = list()

list_oov_src_base = list()
list_oov_src_chat = list()

list_oov_rs_base = list()
list_oov_rs_chat = list()

list_len_rs_base = list()
list_len_rs_chat = list()

list_novel_rs = list()

list_gs_base_icl = list()
list_gs_base_sft = list()
list_gs_chat_icl = list()
list_gs_chat_sft = list()


list_rs = list()
list_sd = list()

for rs in dict_cmp:
  try:
    gen_base_icl,gen_base_sft,gen_chat_icl,gen_chat_sft,rl_base_icl,rl_base_sft,rl_chat_icl,rl_chat_sft = dict_cmp[rs]
    
    sd = dict_src[rs][0]
    
    len_rs_base = len(tokenizer_base.tokenize(rs))
    len_rs_chat = len(tokenizer_chat.tokenize(rs))


    oov_sd_base = dict_src[rs][-2]
    oov_sd_chat = dict_src[rs][-1]

    oov_rs_base = OOV_frac(tokenizer_base,rs)
    oov_rs_chat = OOV_frac(tokenizer_chat,rs)
    
    novel_rs = dict_src[rs][1]
    
    list_rl_base_icl.append(rl_base_icl)
    list_rl_base_sft.append(rl_base_sft)
    list_rl_chat_icl.append(rl_chat_icl)
    list_rl_chat_sft.append(rl_chat_sft)
    
    list_len_rs_base.append(len_rs_base)
    list_len_rs_chat.append(len_rs_chat)
    
    list_gs_base_icl.append(gen_base_icl)
    list_gs_base_sft.append(gen_base_sft)
    list_gs_chat_icl.append(gen_chat_icl)
    list_gs_chat_sft.append(gen_chat_sft)
    
    list_rs.append(rs)
    list_sd.append(sd)
    
    list_novel_rs.append(novel_rs)

    list_oov_rs_base.append(oov_rs_base)
    list_oov_rs_chat.append(oov_rs_chat)
    
    list_oov_src_base.append(oov_sd_base)
    list_oov_src_chat.append(oov_sd_chat)

  except Exception as e:
    print(e)
#     print(rs)
    print(len(dict_cmp[rs]), len(dict_src[rs]))
    pass

In [ ]:
import pandas as pd

df = pd.DataFrame({ 'SD': list_sd,'RS':list_rs, \
                    'GS_Base_ICL': list_gs_base_icl , 'GS_Base_SFT': list_gs_base_sft, \
                    'GS_Chat_ICL': list_gs_chat_icl, 'GS_Chat_SFT': list_gs_chat_sft, \
                    'RS_Len_Base': list_len_rs_base, 'RS_Len_Chat':list_len_rs_chat, \
                    'OOV_RS_Base': list_oov_rs_base, 'OOV_RS_Chat': list_oov_rs_chat, \
                    'Novel_RS': list_novel_rs, \
                    'OOV_SD_Base': list_oov_src_base, 'OOV_SD_Chat': list_oov_src_chat, \
                    'R-L_Base_ICL': list_rl_base_icl , 'R-L_Base_SFT': list_rl_base_sft, \
                    'R-L_Chat_ICL': list_rl_chat_icl, 'R-L_Chat_SFT': list_rl_chat_sft,
                })



In [ ]:
df.to_csv('./CHQ_Llama2_Compare.csv',index=False)

In [ ]:
import pandas as pd
df = pd.read_csv('./CHQ_Llama2_Compare.csv')

In [ ]:
import argparse
from quickumls import QuickUMLS
import pandas as pd
import glob


umls_path = '/Users/gunjanbalde/Documents/QuickUMLS_Files/'
matcher = QuickUMLS(umls_path,similarity_name='cosine',threshold=0.95) 

In [ ]:
from collections import defaultdict
import json
counter_TGT_RS = defaultdict(list)

lines_TGT = df['SD'].to_list()

for idx,abs in enumerate(lines_TGT):
    if idx%10 == 0: print(f'Processed till {idx+1}... ')

    flag = 0
    d = matcher.match(abs, best_match=True, ignore_syntax=False)
    if len(d) == 0:
        counter_TGT_RS[idx].append('happy')
    
    else:
        for l in d:
            counter_TGT_RS[idx].append(l[0]['ngram'])

import re
pattern = r"^[A-Za-z▁]+$"

counter_TGT_RS_Medical_Words = defaultdict(set)

for key,val in counter_TGT_RS.items():
    try:
      print('---------')
      for words in val:
        for word in words.split():
          print(word)
          if re.match(pattern,word.strip()): counter_TGT_RS_Medical_Words[key].add(word.strip())
    except: print("Error in", key,val)

for key,val in counter_TGT_RS_Medical_Words.items():
    print(key,val)
    print('-----------------')

import pickle as pkl
with open('./CHQ_SD_MedicalWords.pkl','wb') as f:
    pkl.dump(counter_TGT_RS_Medical_Words,f)
f.close()


In [ ]:
len(counter_TGT_RS_Medical_Words)

In [ ]:
import pickle as pkl


with open('./CHQ_SD_MedicalWords.pkl','rb') as f:
    counter_TGT_RS_Medical_Words = pkl.load(f)
f.close()

In [ ]:
list_medical_words_splitmorethan1 = list()
list_medical_words_splitmorethan3 = list()

for idx, words in counter_TGT_RS_Medical_Words.items():
    total_words = len(words)
    splitmorethan1 = 0
    splitmorethan2 = 0
    splitmorethan3 = 0
    
    for word in words:
        if len(tokenizer_chat.tokenize(word)) > 1: splitmorethan1 += 1
        if len(tokenizer_chat.tokenize(word)) > 3: splitmorethan3 += 1
    
    list_medical_words_splitmorethan1.append(splitmorethan1/total_words)
    list_medical_words_splitmorethan3.append(splitmorethan3/total_words)


In [ ]:
df['Medical_Words_SplitMoreThan1_SD'] = list_medical_words_splitmorethan1
df['Medical_Words_SplitMoreThan3_SD'] = list_medical_words_splitmorethan3

df.to_csv('./CHQ_Llama2_Compare_WithMedicalWords.csv',index=False)

In [13]:
import pandas as pd
df = pd.read_csv('./CHQ_Llama2_Compare_WithMedicalWords.csv')
df['Novel_RS'].describe(percentiles=[0.1,0.9])

count    150.000000
mean       0.585700
std        0.182671
min        0.090909
10%        0.357143
50%        0.571429
90%        0.833333
max        1.000000
Name: Novel_RS, dtype: float64

In [14]:
df_high_oov = df[df['Novel_RS'] <=0.357143]
df_desc = df_high_oov.drop(columns=['SD', 'RS', 'GS_Base_ICL', 'GS_Base_SFT', 'GS_Chat_ICL', 'GS_Chat_SFT',
       'RS_Len_Base', 'RS_Len_Chat', 'OOV_RS_Base', 'OOV_RS_Chat',
       'OOV_SD_Base', 'OOV_SD_Chat', 'Medical_Words_SplitMoreThan1',
       'Medical_Words_SplitMoreThan3', 'Medical_Words_SplitMoreThan1_SD',
       'Medical_Words_SplitMoreThan3_SD'] )
df_desc.describe(percentiles=[0.5])

,Novel_RS,R-L_Base_ICL,R-L_Base_SFT,R-L_Chat_ICL,R-L_Chat_SFT,CSr__Base_ICL,CSr__Base_SFT,CSr__Chat_ICL,CSr__Chat_SFT
count,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,0.273783,0.398119,0.287364,0.453018,0.249466,0.404762,0.234830,0.268239,0.254230
std,0.077848,0.264844,0.140730,0.220840,0.184576,0.409206,0.276619,0.268450,0.270754
min,0.090909,0.076923,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.292857,0.366071,0.233032,0.437666,0.208711,0.433333,0.174242,0.250000,0.236111
max,0.357143,0.900000,0.583333,0.900000,0.727273,1.000000,0.923077,1.000000,0.750000


High Expert OOV Concentration (Terms Split more than once)
Base        Llama3.1    W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.272727	0.250000    0.279070	0.260870	0.210391	0.263768	0.232946	0.280702

High Expert OOV Concentration (Terms Split more than thrice)
Base        Llama3.1    W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.272727	0.266667    0.279070	0.232558	0.196721	0.250000	0.226415	0.297872

High Expert OOV Concentration (Terms Split more than once) --SD
Base        Llama3.1    W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.267072	0.269160    0.244400	0.230769	0.233032	0.237647	0.224747	0.228390

High Expert OOV Concentration (Terms Split more than thrice) --SD
Base        Llama3.1    W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.280000	0.285714    0.285714	0.263158	0.257143	0.280000	0.254545	0.300000

High Novelty
Base        Llama3.1    W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.215219	0.198361    0.203144	0.165301	0.160645	0.188988	0.167671	0.211646

Low Novelty
Base        Llama3.1    W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.317647	0.324324    0.280000	0.222222	0.236559	0.227273	0.238095	0.303030

## CSr
High Expert OOV Concentration (Terms Split more than once)
Base        Llama3.1    W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.183977	0.163005	0.172201	0.121966	0.104587	0.167962	0.130010	0.146381

High Expert OOV Concentration (Terms Split more than thrice)
Base        Llama3.1    W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.195927	0.186363	0.185171	0.127157	0.121422	0.170255	0.129106	0.169170

High Expert OOV Concentration (Terms Split more than once) --SD
Base        Llama3.1    W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.148352	0.179144	0.138095	0.133333	0.144599	0.156923	0.145503	0.142857

High Expert OOV Concentration (Terms Split more than thrice) --SD
Base        Llama3.1    W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.166667	0.173913	0.153846	0.117647	0.111111	0.166667	0.117647	0.153846

High Novelty
Base        Llama3.1    W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.077510	0.081059	0.090653	0.085247	0.093905	0.078628	0.090186	0.099235

Low Novelty
Base        Llama3.1    W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.250000	0.320000	0.339623	0.181818	0.181818	0.250000	0.210526	0.352941

Long RS (All)
Base        W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.216561	0.208955	0.197802	0.185185	0.212389	0.184874	0.194175

Low Expert OOV Concentration (Terms Split more than once)
Base        W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.280949	0.251761	0.245256	0.244400	0.229670	0.254521	0.262726

Low Expert OOV Concentration (Terms Split more than thrice)
Base        W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.255567	0.232558	0.218750	0.216539	0.208644	0.214286	0.240000

Low Expert OOV Concentration (Terms Split more than once) --SD
Base        W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.233333	0.237288	0.218182	0.210526	0.225806	0.212766	0.22727

Low Expert OOV Concentration (Terms Split more than thrice) --SD
Base        W/O Vocab   MEDVOC      MEDVOC-ABPE Filter      Filter-ABPE Lookup
0.229508	0.208696	0.208333	0.200000	0.200000	0.210526	0.227273



In [ ]:
df_desc = df_desc[df_desc['R-L_Base']<df_desc['R-L_CPT_Lookup']]
df_desc = df_desc[df_desc['R-L_CPT_Without_Vocab']<df_desc['R-L_CPT_Lookup']]

In [ ]:
df_desc_greatest = df_desc.nlargest(int(20), 'R-L_CPT_Lookup')

In [ ]:
for idx,row in df_desc_greatest.iterrows():
    print('------------------------'*8)
    print('+++SD',row['SD'])
    print('+++RS        :',row['RS'])
    print('######################################')
    print('**GS_Base    :',row['GS_Base'])
    print('**GS_W/oVocab:',row['GS_CPT_Without_Vocab'])
    print('**GS_Lookup  :',row['GS_CPT_Lookup'])

In [ ]:
df_long_rs = df[df['RS_Len_Base'] >= 104]
df_long_rs.describe(percentiles=[0.5])

In [ ]:
import pandas as pd
df = pd.read_csv('./BioASQ_Summaries/BioASQ_Llama2_Compare_SelfAdaptBPE_MedicalWords.csv')

In [ ]:
from collections import defaultdict

def getMatchInfo(list_matches):
    dict_concepts = defaultdict(list)
    for match_l in list_matches:
        for match_l_d in match_l:
            if match_l_d["preferred"] == 1: 
                cui = match_l_d['cui'] #the concept-id
                start = match_l_d['start'] 
                end = match_l_d['end']
                n_gram = match_l_d['ngram'].strip() #the surface form

                if '\n' in n_gram : continue
                key = str(start)+'_'+str(end)+'_'+n_gram

                if not cui in dict_concepts[key]: dict_concepts[key].append(cui)
                    
    str_ret = str()
    
    for key,val in dict_concepts.items():
        str_ret += str(key) + ':'
        for c_name in val:
            if not '\n' in c_name:
                str_ret += c_name + '|'
        str_ret += '\n'
    return (str_ret)


import numpy as np
import glob
import os
import pandas as pd

def check_overlap(concepts_ref, concepts_dec,verbose=False):
    list_concepts_ref = concepts_ref.splitlines()
    list_concepts_dec = concepts_dec.splitlines()
    
    cuis_dec, cuis_ref = list(), list()
    
    for line in list_concepts_ref:
        cui_ref = line.split(':')[1].split('|')[:-1]
        cuis_ref.extend(cui_ref)
    
    for line in list_concepts_dec:
        cui_dec = line.split(':')[1].split('|')[:-1]
        cuis_dec.extend(cui_dec)
    
    cuis_ref = set(cuis_ref)
    cuis_dec = set(cuis_dec)
    
    
    common_cuis = cuis_dec.intersection(cuis_ref)
    
    if verbose: print('Common_cuis:', common_cuis)
    
    if len(common_cuis) == 0: return 0.
    
    prec = len(common_cuis)/len(cuis_dec)
    rec = len(common_cuis)/len(cuis_ref)
    
    return ((2*prec*rec)/(prec+rec))
import numpy as np
import glob
import os
import pandas as pd

def check_overlap(concepts_ref, concepts_dec,verbose=False):
    list_concepts_ref = concepts_ref.splitlines()
    list_concepts_dec = concepts_dec.splitlines()
    
    cuis_dec, cuis_ref = list(), list()
    
    for line in list_concepts_ref:
        cui_ref = line.split(':')[1].split('|')[:-1]
        cuis_ref.extend(cui_ref)
    
    for line in list_concepts_dec:
        cui_dec = line.split(':')[1].split('|')[:-1]
        cuis_dec.extend(cui_dec)
    
    cuis_ref = set(cuis_ref)
    cuis_dec = set(cuis_dec)
    
    
    common_cuis = cuis_dec.intersection(cuis_ref)
    
    if verbose: print('Common_cuis:', common_cuis)
    
    if len(common_cuis) == 0: return 0.
    
    prec = len(common_cuis)/len(cuis_dec)
    rec = len(common_cuis)/len(cuis_ref)
    
    return ((2*prec*rec)/(prec+rec))
from numpy.random import seed
from numpy.random import rand
from numpy.random import randint
from numpy import mean
from numpy import median
from numpy import percentile
import numpy as np
def CIEval(f_score):
    dataset = np.array([x[1] for x in f_score])
    max_l = len(dataset)
    scores = list()
    for _ in range(1000):
        # bootstrap sample
        indices = randint(0, max_l, max_l)
        sample = dataset[indices]
        # calculate and store statistic
        statistic = mean(sample)
        scores.append(statistic)

    print('50th percentile (median) = %.4f' % median(scores))
    # calculate 95% confidence intervals (100 - alpha)
    alpha = 5.0
    # calculate lower percentile (e.g. 2.5)
    lower_p = alpha / 2.0
    # retrieve observation at lower percentile
    lower = max(0.0, percentile(scores, lower_p))
#     print('%.1fth percentile = %.4f' % (lower_p, lower))
    # calculate upper percentile (e.g. 97.5)
    upper_p = (100 - alpha) + (alpha / 2.0)
    # retrieve observation at upper percentile
    upper = min(1.0, percentile(scores, upper_p))
#     print('%.1fth percentile = %.4f' % (upper_p, upper))
    print('C.I. Window = %.4f '%max([upper-median(scores),median(scores)-lower]))
for column in ['GS_Base_ICL', 'GS_Base_SFT', 'GS_Chat_ICL', 'GS_Chat_SFT']:

    print(f'--Processing {column}.....')

    f_score = list()
    for row_id in range(df.shape[0]):
        try:                
            dec_sum = df.iloc[row_id][column]
            ref_sum = df.iloc[row_id]['RS']
            ref_sum = ref_sum.split('||')[0]
            
            ref_con = getMatchInfo(matcher.match(ref_sum, best_match=True, ignore_syntax=False))
            dec_con = getMatchInfo(matcher.match(dec_sum, best_match=True, ignore_syntax=False))
            
            # print(f'REF_Sum: {ref_sum}')
            # print(f'DEC_Sum: {dec_sum}')

            # print(check_overlap(ref_con,dec_con))
            f_score.append((row_id,check_overlap(ref_con,dec_con)))
            
        except Exception as e: 
            print(e)
            
#         break
#     break
    df['CSr_'+column[2:]] = [x[1] for x in f_score]
    CIEval(f_score)



In [ ]:
df.to_csv('./CHQ_Llama2_Compare_WithMedicalWords.csv',index=False)

In [ ]:
from scipy.stats import wilcoxon
print(wilcoxon(df_high_oov['R-L_CPT_Without_Vocab'].to_list(),df_high_oov['R-L_CPT_Lookup'].to_list(),alternative='less'), \
wilcoxon(df_low_oov['R-L_CPT_Without_Vocab'].to_list(),df_low_oov['R-L_CPT_Lookup'].to_list(),alternative='less'))

In [ ]:
import re
pattern = r"^[A-Za-z▁]+$"

def OOV_frac(tokenizer,rs):
    tokens = rs.split()
    
    oov_frac = 0
    consider = 0
    oov_list = list()
    for token in tokens:
        if re.match(pattern,token): 
            consider += 1
            if len(tokenizer.tokenize(token)) > 1: 
                oov_frac += 1
                oov_list.append([token,tokenizer.tokenize(token)])
    return oov_frac/consider, oov_list

In [ ]:
list_common_words = []
df_high_oov_better = df_high_oov[df_high_oov['R-L_CPT_Filtering_WithoutAdaptBPE']>df_high_oov['R-L_CPT_Without_Vocab']]
print(df_high_oov_better.shape, df_high_oov.shape)
df_high_oov_better = df_high_oov_better.sort_values(by='R-L_CPT_Filtering_WithoutAdaptBPE',ascending=False)
for idx,row in df_high_oov_better.iterrows():
    rs = row['RS']
    print('++++++++++++', row['R-L_CPT_Filtering_WithoutAdaptBPE'], row['R-L_CPT_Without_Vocab'],row['Novel_RS'],row['OOV_RS_Base'],row['OOV_RS_Filtering_WithoutAdaptBPE'],row['RS_Len_Base'],row['RS_Len_Filtering_WithoutAdaptBPE'])
    print('-- SD: ',row['SD'])
    print('-- RS                               :',row['RS'])
    print('***')
    print('-- GS_CPT_Without_Vocab             :',row['GS_CPT_Without_Vocab'])
    print('-- GS_CPT_Without_Filtering_AdaptBPE:',row['GS_CPT_Filtering_WithoutAdaptBPE'])
    print('-- OOV Base Tokenizer               :', OOV_frac(tokenizer,rs)) 
    print('-- Lookup-1000                      :',OOV_frac(tokenizer_lookup,rs))
    common_words = 0
    for token1 in  row['GS_CPT_Filtering_WithoutAdaptBPE'].split():
        if token1 in row['GS_CPT_Without_Vocab']: common_words+=1
        
    common_words /= len(row['GS_CPT_Filtering_WithoutAdaptBPE'].split()) 
    
    list_common_words.append(common_words)
    print('-- Common Words Frac               :', common_words)
    

In [ ]:
df_high_oov_worst = df_high_oov[df_high_oov['R-L_CPT_Filtering_WithoutAdaptBPE']<=df_high_oov['R-L_CPT_Without_Vocab']]
list_common_words_worst = []
print(df_high_oov_worst.shape, df_high_oov.shape)
df_high_oov_worst = df_high_oov_worst.sort_values(by='R-L_CPT_Filtering_WithoutAdaptBPE')
for idx,row in df_high_oov_worst.iterrows():
    rs = row['RS']
    print('++++++++++++', row['R-L_CPT_Filtering_WithoutAdaptBPE'], row['R-L_CPT_Without_Vocab'],row['Novel_RS'],row['OOV_RS_Base'],row['OOV_RS_Filtering_WithoutAdaptBPE'],row['RS_Len_Base'],row['RS_Len_Filtering_WithoutAdaptBPE'])
    print('-- SD: ',row['SD'])
    print('-- RS                               : ',row['RS'])
    print('***')
    print('-- GS_CPT_Without_Vocab             :',row['GS_CPT_Without_Vocab'])
    print('-- GS_CPT_Without_Filtering_AdaptBPE:',row['GS_CPT_Filtering_WithoutAdaptBPE'])
    print('-- OOV Base Tokenizer               :', OOV_frac(tokenizer,rs)) 
    print('-- Filtering w/o AdaptBPE           :',OOV_frac(tokenizer_filtering,rs))
    common_words = 0
    for token1 in  row['GS_CPT_Filtering_WithoutAdaptBPE'].split():
        if token1 in row['GS_CPT_Without_Vocab']: common_words+=1
        
    common_words /= len(row['GS_CPT_Filtering_WithoutAdaptBPE'].split()) 
    
    list_common_words_worst.append(common_words)
    print('-- Common words Frac                :', common_words)

In [ ]:
df.describe(percentiles=[0.1,0.25,0.5,0.75,0.9])

In [ ]:
df_high_oov = df[df['OOV_RS_Lookup']>=0.27]
df_high_oov.describe()

df_long_rs = df[df['RS_Len_Lookup']>=93]
df_long_rs.describe()

In [ ]:
df_high_oov.describe()

In [ ]:
Long-RS:
Base > CPT_Filter_W/OABPE > CPT_W/O_Vocab > CPT_Lookup > CPT_Filter_AdaptBPE
0.219003 > 0.213338 > 0.208825 > 0.205657 > 0.185030

High-OOV:
CPT_Lookup > Base > CPT_Filter_AdaptBPE > CPT_Filter_W/OABPE > CPT_W/O_Vocab
0.263158 > 0.258094 > 0.255865 > 0.251984 > 0.245526

In [ ]:
df_high_oov = df[df['OOV_RS_Filtering_WithoutAdaptBPE']>=0.25]
df_high_oov.describe()

High-OOV:
Lookup25.59 > W/O_Vocab25.00 > Base24.83 > Filter_W/OABPE22.04 > Filter_ABPE20.97


In [ ]:
df_long_rs = df[df['RS_Len_Filtering_WithoutAdaptBPE']>=89]
df_long_rs.describe()

Long-RS:
Base20.64 > Lookup20.15 > W/O_Vocab19.33 > Filter_W/OABPE18.90 > Filter_ABPE18.27

In [ ]:
list_common_words = []
df_high_oov_better = df_high_oov[df_high_oov['R-L_CPT_Lookup']>df_high_oov['R-L_CPT_Without_Vocab']]
print(df_high_oov_better.shape, df_high_oov.shape)
df_high_oov_better = df_high_oov_better.sort_values(by='R-L_CPT_Lookup',ascending=False)
for idx,row in df_high_oov_better.iterrows():
    rs = row['RS']
    print('++++++++++++', row['R-L_CPT_Lookup'], row['R-L_CPT_Without_Vocab'],row['Novel_RS'],row['OOV_RS_Base'],row['OOV_RS_Lookup'],row['RS_Len_Base'],row['RS_Len_Lookup'])
    print('-- SD: ',row['SD'])
    print('-- RS                               :',row['RS'])
    print('***')
    print('-- GS_CPT_Without_Vocab             :',row['GS_CPT_Without_Vocab'])
    print('-- GS_CPT_Without_Filtering_AdaptBPE:',row['GS_CPT_Lookup'])
    print('-- OOV Base Tokenizer               :', OOV_frac(tokenizer,rs)) 
    print('-- Lookup-1000                      :',OOV_frac(tokenizer_lookup,rs))
    common_words = 0
    for token1 in  row['GS_CPT_Lookup'].split():
        if token1 in row['GS_CPT_Without_Vocab']: common_words+=1
        
    common_words /= len(row['GS_CPT_Lookup'].split()) 
    
    list_common_words.append(common_words)
    print('-- Common Words Frac               :', common_words)
    

In [ ]:
list_common_words = []
df_high_oov_worst = df_high_oov[df_high_oov['R-L_CPT_Lookup']<=df_high_oov['R-L_CPT_Without_Vocab']]
print(df_high_oov_worst.shape, df_high_oov.shape)
df_high_oov_better = df_high_oov_worst.sort_values(by='R-L_CPT_Lookup',ascending=False)
for idx,row in df_high_oov_worst.iterrows():
    rs = row['RS']
    print('++++++++++++', row['R-L_CPT_Lookup'], row['R-L_CPT_Without_Vocab'],row['Novel_RS'],row['OOV_RS_Base'],row['OOV_RS_Lookup'],row['RS_Len_Base'],row['RS_Len_Lookup'])
    print('-- SD: ',row['SD'])
    print('-- RS                               :',row['RS'])
    print('***')
    print('-- GS_CPT_Without_Vocab             :',row['GS_CPT_Without_Vocab'])
    print('-- GS_CPT_Without_Filtering_AdaptBPE:',row['GS_CPT_Lookup'])
    print('-- OOV Base Tokenizer               :', OOV_frac(tokenizer,rs)) 
    print('-- Lookup-1000                      :',OOV_frac(tokenizer_lookup,rs))
    common_words = 0
    for token1 in  row['GS_CPT_Lookup'].split():
        if token1 in row['GS_CPT_Without_Vocab']: common_words+=1
        
    common_words /= len(row['GS_CPT_Lookup'].split()) 
    
    list_common_words.append(common_words)
    print('-- Common Words Frac               :', common_words)
    

In [ ]:
import pandas as pd
df_BioASQ = pd.read_csv('./BioASQ_Summaries/BioASQ_Llama2_Compare_SelfAdaptBPE_MedicalWords_WithLlama3.csv')
df_BioASQ